Following code computes the $\operatorname{Crit}^w$ for the cone generated by $(1,0,0),(0,1,0),(1,1,2)$ with $w=(1,1,1),(1,2,2)$.

In [3]:
import itertools as _itertools
from math import ceil
from sage.all import *

def diameter_D(sigmadual):
    rays = [vector(r) for r in list(sigmadual.rays())]
    n = len(rays)
    if n == 0:
        return 0
    combos = []
    for coeffs in _itertools.product([0, 1], repeat=n):
        v = sum(coeffs[i] * rays[i] for i in range(n))
        combos.append(v)
    max_dist = 0.0
    for i in range(len(combos)):
        for j in range(i + 1, len(combos)):
            d = (combos[i] - combos[j]).norm()
            if d > max_dist:
                max_dist = float(d)
    return int(ceil(max_dist))

def minimal_face_containing_w(sigma, w):
    dim_sigma = sigma.dimension()
    for k in range(1, dim_sigma + 1):
        for F in sigma.faces(k):
            if F.contains(w):
                return F
    return None

def compute_weight(vi, w, D, cone_vectors):
    from itertools import product
    n = len(vi)
    minus_vi = -vi
    delta = Cone(cone_vectors)
    deltaDual = delta.dual()
    Hw = [vector(h) for h in deltaDual.Hilbert_basis()]
    H_non_orth = [h for h in Hw if w.dot_product(h) != 0]
    if not H_non_orth:
        return matrix(QQ, n, 0)
    vals_vi = [minus_vi.dot_product(h) for h in H_non_orth]
    c = min(vals_vi)
    valid_points = []
    for coords in product(range(-D, D + 1), repeat=n):
        u = vector(coords)
        if u.is_zero() or not deltaDual.contains(u):
            continue
        vali = minus_vi.dot_product(u)
        valw = w.dot_product(u)
        if vali < c and valw == 0:
            valid_points.append(u)
    if not valid_points:
        return matrix(QQ, n, 0)
    Critiw = Matrix(QQ, n, len(valid_points), lambda i, j: valid_points[j][i])
    return Critiw

def criticalArrow(vecsList, w):
    sigma = Cone(vecsList)
    D = diameter_D(sigma.dual())
    print("D =", D)
    mu = minimal_face_containing_w(sigma, w)
    if mu is None:
        return 0
    mu_dim = mu.dimension()
    mu_rays = [vector(r) for r in mu.rays()]
    Critw = matrix(QQ, len(vecsList[0]), 0)
    for vi in mu_rays:
        other_rays = [v for v in vecsList if v != vi]
        Critiw = compute_weight(vi, w, D, other_rays + [-w])
        print("vi =", vi)
        print("Critiw =")
        print(Critiw)
        Critw = Critw.augment(Critiw)
        print("Critw =")
        print(Critw)
    if Critw.ncols() == 0:
        return 0
    Vw = Critw.column_space()
    mu_ray_matrix = matrix(QQ, mu_rays).transpose()
    mu_orth = mu_ray_matrix.left_kernel()
    dim_arr = Vw.dimension() - Vw.intersection(mu_orth).dimension()
    return mu_dim - dim_arr

In [4]:
myVecsList = [(vector([1,0,0]), vector([0,1,0]), vector([1,1,2]))]

for vecsList in myVecsList:
    print("generators:", [tuple(v) for v in vecsList])
    sigma = Cone(vecsList)
    exceptional = [vector([1,1,1]),vector([1,2,2])]
    for w in exceptional:
        dim_tau = criticalArrow(vecsList, w)
        print(f" w={tuple(w)}, dim_tau={dim_tau}")

generators: [(1, 0, 0), (0, 1, 0), (1, 1, 2)]
D = 5
vi = N(1, 0, 0)
Critiw =
[-1 -1]
[ 0  1]
[ 1  0]
Critw =
[-1 -1]
[ 0  1]
[ 1  0]
vi = N(0, 1, 0)
Critiw =
[ 0  1]
[-1 -1]
[ 1  0]
Critw =
[-1 -1  0  1]
[ 0  1 -1 -1]
[ 1  0  1  0]
vi = N(1, 1, 2)
Critiw =
[ 0  1]
[ 1  0]
[-1 -1]
Critw =
[-1 -1  0  1  0  1]
[ 0  1 -1 -1  1  0]
[ 1  0  1  0 -1 -1]
 w=(1, 1, 1), dim_tau=1
D = 5
vi = N(0, 1, 0)
Critiw =
[ 2  4]
[ 0  0]
[-1 -2]
Critw =
[ 2  4]
[ 0  0]
[-1 -2]
vi = N(1, 1, 2)
Critiw =
[ 2  4]
[ 0  0]
[-1 -2]
Critw =
[ 2  4  2  4]
[ 0  0  0  0]
[-1 -2 -1 -2]
 w=(1, 2, 2), dim_tau=2
